In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("spark_on_docker").getOrCreate()

In [5]:
!wget -q https://github.com/jupyter/docker-stacks/blob/d990a62010aededcda836196c4b04efece7f838f/README.md 


In [6]:
textFile = spark.read.text("README.md")
rows_with_jupyter = textFile.filter(textFile.value.contains("Jupyter")).count()

print("There's %d rows with the word Jupyter."%rows_with_jupyter)

There's 1 rows with the word Jupyter.


In [7]:
spark

In [1]:
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.context import SparkContext
import os
from IPython.display import display, clear_output
import time

spark = SparkSession.builder \
.master("local[*]") \
.appName("spark_on_docker") \
.config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0') \
.getOrCreate()


In [2]:
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka_container:9093").option("subscribe", "LazerScan").option("startingOffsets", "earliest").option("maxOffsetsPerTrigger", 100).load()

print(str(df.isStreaming))
df.printSchema()

True
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [3]:
query = df.withWatermark("timestamp", "3 minutes") \
    .writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("LazerScanQuery1") \
    .start()

In [4]:
while True:
    clear_output(wait=True)
    display(query.status)
    display(spark.sql('SELECT CAST(value AS STRING) FROM LazerScanQuery1').show(200,truncate=False))
    display(query.lastProgress)
    display(query.status)
    display(query.isActive)
    time.sleep(1)



{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

+-----+
|value|
+-----+
+-----+



None

{'id': '78ce3568-b593-435c-a5d8-9150d0387768',
 'runId': '361da550-5cb3-4dab-9348-52ed2711c4e8',
 'name': 'LazerScanQuery1',
 'timestamp': '2023-10-17T15:47:19.483Z',
 'batchId': 1,
 'numInputRows': 0,
 'inputRowsPerSecond': 0.0,
 'processedRowsPerSecond': 0.0,
 'durationMs': {'latestOffset': 1, 'triggerExecution': 1},
 'eventTime': {'watermark': '1970-01-01T00:00:00.000Z'},
 'stateOperators': [],
 'sources': [{'description': 'KafkaV2[Subscribe[LazerScan]]',
   'startOffset': {'LazerScan': {'0': 0}},
   'endOffset': {'LazerScan': {'0': 0}},
   'latestOffset': {'LazerScan': {'0': 0}},
   'numInputRows': 0,
   'inputRowsPerSecond': 0.0,
   'processedRowsPerSecond': 0.0,
   'metrics': {'avgOffsetsBehindLatest': '0.0',
    'maxOffsetsBehindLatest': '0',
    'minOffsetsBehindLatest': '0'}}],
 'sink': {'description': 'MemorySink', 'numOutputRows': 0}}

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

True

KeyboardInterrupt: 